In [170]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.decomposition import PCA
from xgboost import XGBClassifier,XGBRegressor
from sklearn.linear_model import LinearRegression


In [171]:
df = pd.read_csv('Smart_Farming_Crop_Yield_2024.csv')
df.head()

,farm_id,region,crop_type,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,irrigation_type,...,sowing_date,harvest_date,total_days,yield_kg_per_hectare,sensor_id,timestamp,latitude,longitude,NDVI_index,crop_disease_status
0,FARM0001,North India,Wheat,35.95,5.99,17.79,75.62,77.03,7.27,NaN,...,2024-01-08,2024-05-09,122,4408.07,SENS0001,2024-03-19,14.970941,82.997689,0.63,Mild
1,FARM0002,South USA,Soybean,19.74,7.24,30.18,89.91,61.13,5.67,Sprinkler,...,2024-02-04,2024-05-26,112,5389.98,SENS0002,2024-04-21,16.613022,70.869009,0.58,NaN
2,FARM0003,South USA,Wheat,29.32,7.16,27.37,265.43,68.87,8.23,Drip,...,2024-02-03,2024-06-26,144,2931.16,SENS0003,2024-02-28,19.503156,79.068206,0.80,Mild
3,FARM0004,Central USA,Maize,17.33,6.03,33.73,212.01,70.46,5.03,Sprinkler,...,2024-02-21,2024-07-04,134,4227.80,SENS0004,2024-05-14,31.071298,85.519998,0.44,NaN
4,FARM0005,Central USA,Cotton,19.37,5.92,33.86,269.09,55.73,7.93,NaN,...,2024-02-05,2024-05-20,105,4979.96,SENS0005,2024-04-13,16.568540,81.691720,0.84,Severe


In [172]:
df = df.drop(columns=['farm_id','sensor_id' ], axis=1)

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   region                500 non-null    object 
 1   crop_type             500 non-null    object 
 2   soil_moisture_%       500 non-null    float64
 3   soil_pH               500 non-null    float64
 4   temperature_C         500 non-null    float64
 5   rainfall_mm           500 non-null    float64
 6   humidity_%            500 non-null    float64
 7   sunlight_hours        500 non-null    float64
 8   irrigation_type       350 non-null    object 
 9   fertilizer_type       500 non-null    object 
 10  pesticide_usage_ml    500 non-null    float64
 11  sowing_date           500 non-null    object 
 12  harvest_date          500 non-null    object 
 13  total_days            500 non-null    int64  
 14  yield_kg_per_hectare  500 non-null    float64
 15  timestamp             5

In [5]:
df.fillna('None', inplace=True)
df['sowing_date'] = pd.to_datetime(df['sowing_date'], format='%Y-%m-%d', errors='coerce')
df['sowing_month'] = df['sowing_date'].dt.month
# df['sowing_quarter'] = df['sowing_date'].dt.quarter
df['harvest_date'] = pd.to_datetime(df['harvest_date'], format='%Y-%m-%d', errors='coerce')
df['harvest_month'] = df['harvest_date'].dt.month
# df['harvest_quarter'] = df['harvest_date'].dt.quarter
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d', errors='coerce')
df['timestamp_month'] = df['timestamp'].dt.month
# df['timestamp_quarter'] = df['timestamp'].dt.quarter
df = df.drop(columns=['sowing_date', 'harvest_date', 'timestamp'], axis=1)
df.head()

,region,crop_type,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,irrigation_type,fertilizer_type,pesticide_usage_ml,total_days,yield_kg_per_hectare,latitude,longitude,NDVI_index,crop_disease_status,sowing_month,harvest_month,timestamp_month
0,North India,Wheat,35.95,5.99,17.79,75.62,77.03,7.27,None,Organic,6.34,122,4408.07,14.970941,82.997689,0.63,Mild,1,5,3
1,South USA,Soybean,19.74,7.24,30.18,89.91,61.13,5.67,Sprinkler,Inorganic,9.60,112,5389.98,16.613022,70.869009,0.58,None,2,5,4
2,South USA,Wheat,29.32,7.16,27.37,265.43,68.87,8.23,Drip,Mixed,15.26,144,2931.16,19.503156,79.068206,0.80,Mild,2,6,2
3,Central USA,Maize,17.33,6.03,33.73,212.01,70.46,5.03,Sprinkler,Organic,25.80,134,4227.80,31.071298,85.519998,0.44,None,2,7,5
4,Central USA,Cotton,19.37,5.92,33.86,269.09,55.73,7.93,None,Mixed,25.65,105,4979.96,16.568540,81.691720,0.84,Severe,2,5,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   region                500 non-null    object 
 1   crop_type             500 non-null    object 
 2   soil_moisture_%       500 non-null    float64
 3   soil_pH               500 non-null    float64
 4   temperature_C         500 non-null    float64
 5   rainfall_mm           500 non-null    float64
 6   humidity_%            500 non-null    float64
 7   sunlight_hours        500 non-null    float64
 8   irrigation_type       500 non-null    object 
 9   fertilizer_type       500 non-null    object 
 10  pesticide_usage_ml    500 non-null    float64
 11  total_days            500 non-null    int64  
 12  yield_kg_per_hectare  500 non-null    float64
 13  latitude              500 non-null    float64
 14  longitude             500 non-null    float64
 15  NDVI_index            5

In [176]:
categorical_columns = ["crop_type", "region", "irrigation_type", "fertilizer_type", "crop_disease_status"]

df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   soil_moisture_%               500 non-null    float64
 1   soil_pH                       500 non-null    float64
 2   temperature_C                 500 non-null    float64
 3   rainfall_mm                   500 non-null    float64
 4   humidity_%                    500 non-null    float64
 5   sunlight_hours                500 non-null    float64
 6   pesticide_usage_ml            500 non-null    float64
 7   total_days                    500 non-null    int64  
 8   yield_kg_per_hectare          500 non-null    float64
 9   latitude                      500 non-null    float64
 10  longitude                     500 non-null    float64
 11  NDVI_index                    500 non-null    float64
 12  sowing_month                  500 non-null    int32  
 13  harve

In [ ]:
correlations = df.corr(numeric_only=True)["yield_kg_per_hectare"].sort_values(ascending=False)

print(correlations)

In [178]:
x = df.drop(['yield_kg_per_hectare','total_days'], axis=1).values
y = df['yield_kg_per_hectare'].values

In [179]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [180]:
scale = StandardScaler()
x_train = scale.fit_transform(x_train)
x_test = scale.transform(x_test)

In [181]:
# poly = PolynomialFeatures(degree=3)
# x_train = poly.fit_transform(x_train)
# x_test = poly.transform(x_test)

In [182]:
le = LinearRegression()
le.fit(x_train, y_train)
y_pred = le.predict(x_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))

Mean Squared Error: 1512105.1143569646
R^2 Score: -0.09488629526330805


In [183]:
rf = RandomForestRegressor(n_estimators=100,max_depth=3, random_state=42)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))

Mean Squared Error: 1446048.844870574
R^2 Score: -0.04705621818058803


In [184]:
xgb = XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=42)
xgb.fit(x_train, y_train)
y_pred_xgb = xgb.predict(x_test)
print("XGBoost Mean Squared Error:", mean_squared_error(y_test, y_pred_xgb))
print("XGBoost R^2 Score:", r2_score(y_test, y_pred_xgb))

XGBoost Mean Squared Error: 1585286.5978976232
XGBoost R^2 Score: -0.1478756031063535


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

params = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.9, 1.0]
}

xgb = XGBRegressor(random_state=42)
grid = GridSearchCV(xgb, param_grid=params, cv=3, scoring="r2", n_jobs=-1)
grid.fit(x_train, y_train)

print("Best Params:", grid.best_params_)
print("Best R^2 Score:", grid.best_score_)